In [10]:
import streamlit as st
from langchain_community.llms import Ollama
from src.retriever import load_hybrid_retriever
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

In [21]:
load_dotenv()  # take environment variables from .env file
# llm = Ollama(model="llama3", base_url="http://localhost:11434")
from langchain_community.llms import HuggingFaceHub
import os

import os
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    api_key=os.environ["GOOGLE_API_KEY"]
)


#  Make sure to set this in Hugging Face Spaces Secrets
# hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# from langchain_huggingface import HuggingFaceEndpoint

# llm = HuggingFaceEndpoint(
#     repo_id="meta-llama/Llama-3.1-8B",  # You can swap with another model
#     huggingfacehub_api_token=hf_token,
#     temperature=0.3,
#     max_new_tokens=1500
# )


# Load prebuilt retriever
retriever = load_hybrid_retriever()

# Prompt
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a legal AI assistant. Use the given context from IPC/Constitution to answer the question.
If the answer is not in the context, say you don't know.

Context:
{context}

Question:
{question}

Answer concisely, but accurately in legal terms.
"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)


In [22]:
docs = retriever.get_relevant_documents("what is the punishment for murder under IPC?")
print(docs)


[Document(id='13d05b61-9c2b-418a-b05e-6f32c3387f13', metadata={'producer': 'Microsoft® Office Word 2007', 'creator': 'Microsoft® Office Word 2007', 'creationdate': 'D:20170823054222', 'author': 'Admin', 'moddate': 'D:20170823054222', 'source': 'data\\IPC.pdf', 'total_pages': 112, 'page': 71, 'page_label': '72'}, page_content='72 \n \n302. Punishment for murder .—Whoever commits murder sha ll be punished with death or \n1[imprisonment for life], and shall also be liable to fine. \n303. Punishment for murder by life -convict.—Whoever, being under sentence of 1[imprisonment \nfor life], commits murder, shall be punished with death. \n304. Punishment for culpable homi cide not amounting to murder .—Whoever commits culpable \nhomicide not amounting to murder, shall be punished with 1[imprisonment for life], or imprisonment of'), Document(id='cf9a19dd-c9cc-44c9-a846-e1069031a76d', metadata={'producer': 'Microsoft® Office Word 2007', 'creator': 'Microsoft® Office Word 2007', 'creationdate': '

In [23]:
query = "murder?"

In [24]:
result = qa_chain({"query": query})

In [25]:
result

{'query': 'murder?',
 'result': 'Section 302 outlines the punishment for murder as death or imprisonment for life, and a fine.  Section 303 addresses murder committed by a life convict, prescribing the death penalty.  Section 304 details the punishment for culpable homicide not amounting to murder as life imprisonment or imprisonment of another term.'}

In [26]:
for doc in result.get("source_documents", []):
        st.write(f"- {doc.metadata.get('source', 'Unknown')}")

In [27]:
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEndpoint

# Load environment variables
load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# Initialize LLM
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B",  # Swap if needed
    huggingfacehub_api_token=hf_token,
    temperature=0.3,
    max_new_tokens=512
)

# Load retriever (make sure it has your IPC documents indexed)
retriever = load_hybrid_retriever()

# Define clean prompt
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a legal AI assistant. Use the given context from IPC/Constitution to answer the question.
If the answer is not in the context, say you don't know.

Context:
{context}

Question:
{question}

Answer concisely, but accurately in legal terms.
"""
)

# Wrap retriever in RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",  # Use 'stuff' to concatenate all retrieved docs
    chain_type_kwargs={"prompt": prompt}
)

# Ask a question
question = "What is the punishment for murder under IPC?"

# Option 1: Using the QA chain (recommended)
answer = qa_chain.run(question)
print("Answer via RetrievalQA:", answer)

# Option 2: Test LLM directly with cleaned context
docs = retriever.get_relevant_documents(question)
context = " ".join([doc.page_content.replace("\n", " ") for doc in docs])
llm_input = f"Context:\n{context}\n\nQuestion:\n{question}\nAnswer concisely, but accurately in legal terms."
direct_answer = llm(llm_input)
print("Direct LLM answer:", direct_answer)


Answer via RetrievalQA: 
Direct LLM answer: 


In [28]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.environ.get("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    api_key=api_key
)

response = llm.invoke("Explain IPC Section 420 in simple words")
print(response.content)


IPC Section 420, in simple words, deals with **cheating and dishonesty to induce someone to deliver property**.

It means if someone tricks you into giving them something valuable (money, goods, etc.) by lying or using other dishonest means, they've committed an offense under this section.  The key is the *deception* and the resulting *unlawful gain* for the cheater and *unlawful loss* for the victim.
